# 研究有些股票开始上涨的原因.
本来想用 祥和实业 从20200728 开始的这波涨势为例子进行研究, 但是考虑到 daily data 的原因.是否可以找到一个从 20200810 开始上涨的 股票.
### 载入数据

In [1]:
import math
import datetime
import multiprocessing as mp
import tushare as ts
import os
import numpy as np
import pandas as pd
import datetime
import arrow
p = print
# tushare 的初始化
token = '42e0612206685cd9bcd250cc5a3ef3931149303bc1d512c261174b5d'
pro = ts.pro_api(token)
# 获取当前上市公司列表，包括股票代码，注册地，行业，上市时间等数据
basic = pro.stock_basic(list_status = 'L')
# 这个 cell 是必须要载入的

In [2]:
# 检索所有股票, 找到一只,从 20200810 开始一直在上涨的股票.abs
upstart_time = '20200810'
select_stocklist = dict()
for one_tscode in basic['ts_code']:
    stock_data_p = ts.pro_bar(api = pro, ts_code = one_tscode, start_date = upstart_time, adj = 'qfq')
    #p(stock_data_p)
    # 因为从 20200810 到现在一共只有 10 个交易日, 则我们用下面这个判断规则来筛选:
    # 1, 上涨的天数,大于 8 ?
    if sum(stock_data_p['pct_chg'] > 0) > 8:
        if sum(stock_data_p['pct_chg'] > 3) > 5:
            select_stocklist[stock_data_p.iloc[0]['ts_code']] = stock_data_p


In [3]:
select_stocklist.keys()
# 002919 感觉上是符合要求的.

dict_keys(['002108.SZ', '002335.SZ', '002716.SZ', '002919.SZ', '002991.SZ', '300094.SZ', '300312.SZ', '300313.SZ', '300647.SZ', '300778.SZ', '300859.SZ', '600158.SH', '600226.SH', '600568.SH', '601456.SH', '603076.SH', '603167.SH', '603303.SH'])

In [2]:
# 用程序筛选:
# 多天盘整,然后向上突破均线后形成上涨趋势.
# 观察 300204 的数据: 从20200619 到 20200701 是一个调整期,
# 而从 20200703 开始, 突破了均线后, 形成了一个上涨势头.
study_starttime = '20200601'
oneselect_stock = basic[basic['symbol'] == '300204']['ts_code'].iloc[0]
oneselect_stockdata = ts.pro_bar(api = pro, ts_code = oneselect_stock, start_date = study_starttime, adj = 'qfq')

In [9]:
# 要把数据的时间格式改一下.
oneselect_stockdata['trans_time'] = pd.to_datetime(oneselect_stockdata['trade_date'])
#print(oneselect_stockdata)
# 设定一个 检查的时间长度 len_Maverage
len_Maverage = 4
len_select = int(len_Maverage * 1.5)
# 找一段 ,在这一段里找跌幅绝大多数在 2% 以内, 偶尔有两个会在 3%左右.这个限制用 in2_limit 表示
in2_limit = 0.7
in3_limit = 0.95
###############################################################################
# 选择一个超过这个时间长度的调整期
for i_len in range(len_select + 1, len(oneselect_stockdata)):
    analysis_data = oneselect_stockdata.iloc[-i_len:-(i_len- len_select), :]
    up2_now = sum(abs(analysis_data['pct_chg']) <= 2)/len(analysis_data)
    if up2_now > in2_limit :
        up3_now = sum(abs(analysis_data['pct_chg']) <= 3)/len(analysis_data)
        if up3_now > in3_limit :
            # 这里是检验,让开盘收盘很接近.
            close_beyond_open = []
            for i in range(len(analysis_data)):
                temp_data = (analysis_data.iloc[i,2] - analysis_data.iloc[i,5])/analysis_data.iloc[i,5]
                close_beyond_open.append(round(temp_data,5))
            up5_sum = sum(abs(np.array(close_beyond_open)) > 0.05)
            if up5_sum == 0 :
                # 到这里为止,已经可以从数据例子中: 观察 300204 的数据: 从20200619 到 20200701 是一个调整期,,在这个例子中准确额找到 20200619 这个日期,
                # 那么... 下一步是 ,在之后的一天 向上突破均线.(目前设定为三天内)



                break
p(analysis_data)
p('~~~~~~~~~~~~~~~~~~~')
p(i_len)



      ts_code trade_date   open   high    low  close  pre_close  change  \
41  300204.SZ   20200630  11.09  11.24  11.03  11.16      11.17   -0.01   
42  300204.SZ   20200629  11.31  11.56  11.13  11.17      10.98    0.19   
43  300204.SZ   20200624  11.16  11.17  10.88  10.98      11.16   -0.18   
44  300204.SZ   20200623  11.30  11.42  11.08  11.16      11.24   -0.08   
45  300204.SZ   20200622  11.24  11.36  11.14  11.24      11.24    0.00   
46  300204.SZ   20200619  11.37  11.47  11.23  11.24      11.54   -0.30   

    pct_chg       vol     amount trans_time  
41  -0.0895  31498.10  35105.126 2020-06-30  
42   1.7304  50637.52  57334.382 2020-06-29  
43  -1.6129  24718.68  27203.880 2020-06-24  
44  -0.7117  27886.50  31230.235 2020-06-23  
45   0.0000  26644.82  29972.600 2020-06-22  
46  -2.5997  41355.75  46785.171 2020-06-19  
~~~~~~~~~~~~~~~~~~~
20


In [2]:

# 目前这个算法在使用
###############################################################################

def find_adjustperiod(oneselect_stockdata, # 所选择的股票数据,dataframe
                        len_Maverage = 4, # 设定一个 检查的时间长度
                      in2_limit = 0.7,in3_limit = 0.95,
                      # 找一段 ,在这一段里找跌幅绝大多数在 2% 以内, 偶尔有两个会在 3%左右.这个限制用 in2_limit 表示
                      give_test_time = 8 # 给后续检验留下的时间窗口
                     ):
    # selectdata_list 用于储存最后选择出来的股票时间段.
    selectdata_list = []
    # 要把数据的时间格式改一下.
    oneselect_stockdata['trans_time'] = pd.to_datetime(oneselect_stockdata['trade_date'])
    len_select = int(len_Maverage * 1.5)
    #######################################################################################
    # 减去 give_test_time 是为了给后续验证留下时间段.
    # 这里为了让循环,当遇到对的选择时,进行跳过,所以把 for 改成 while
    i_len = len_select + 1
    #for i_len in range(len_select + 1, len(oneselect_stockdata) - give_test_time):
    while i_len <= len(oneselect_stockdata) - give_test_time:
        analysis_data = oneselect_stockdata.iloc[-i_len:-(i_len- len_select), :]
        up2_now = sum(abs(analysis_data['pct_chg']) <= 2)/len(analysis_data)
        if up2_now > in2_limit :
            up3_now = sum(abs(analysis_data['pct_chg']) <= 3)/len(analysis_data)
            if up3_now > in3_limit :
                # 这里是检验,让开盘收盘很接近.
                close_beyond_open = []
                for i in range(len(analysis_data)):
                    temp_data = (analysis_data.iloc[i,2] - analysis_data.iloc[i,5])/analysis_data.iloc[i,5]
                    close_beyond_open.append(round(temp_data,5))
                up5_sum = sum(abs(np.array(close_beyond_open)) > 0.05)
                if up5_sum == 0 :
                    # 到这里为止,已经可以从数据例子中: 观察 300204 的数据: 从20200619 到 20200701 是一个调整期,,在这个例子中准确额找到 20200619 这个日期,
                    # 那么... 下一步是 ,在之后的一天 向上突破均线.(目前设定为三天内)
                    selectdata_list.append(analysis_data)
                    # 选择到一个对的向后跳过
                    i_len += len_Maverage + give_test_time + 3
                else :
                    i_len += 1
            else:
                i_len += 1
        else :
            i_len += 1
    return selectdata_list
    
#test_onefind = find_adjustperiod(oneselect_stockdata)
#print(test_onefind)

NameError: name 'oneselect_stockdata' is not defined

In [33]:
# 这里选出来的时间段的最后一天是: 20200630 , 逐一检验后续日期中是否有可以超过均线的天数.
# 检验的日期长度设定为: test_len
test_len = 3 # 检验后续三天的

#######################
import arrow
last_date = analysis_data[0].iloc[-1]['trade_date']
last_date_arrow = arrow.get(last_date, 'YYYYMMDD')
print(last_date_arrow)

# 获得 average line 的起始时间: 留了足够的时间长度, 获得 50 日平均线应该也是够了.
startdate_averline = last_date_arrow.shift(days = -30 * 2)
str_startdate_averline = startdate_averline.format('YYYYMMDD')
print(str_startdate_averline)

# 获得平均线数据:
# 均线, 取得上面数据的均线:
# vol : 成交量; amount : 成交额; ma_v_5 : 是五日平均的成交量.
# 这个五日平均线是自动计算,所以要获得 0701 的平均线结果 要提前足够的天数.
average_line = ts.pro_bar(api = pro,ts_code = '000001.SZ', start_date = str_startdate_averline,  ma=[5, 10,20])

average_line


2019-01-07T00:00:00+00:00
20181108


,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount,ma5,ma_v_5,ma10,ma_v_10,ma20,ma_v_20
0,000001.SZ,20200826,14.60,14.61,14.28,14.37,14.60,-0.23,-1.5753,734117.72,1057274.169,14.494,994041.150,14.656,1294990.030,14.2360,1.426989e+06
1,000001.SZ,20200825,14.56,14.69,14.46,14.60,14.46,0.14,0.9682,748320.22,1090756.854,14.640,1131403.228,14.657,1381259.428,14.1945,1.466262e+06
2,000001.SZ,20200824,14.50,14.71,14.41,14.46,14.45,0.01,0.0692,919448.86,1338031.969,14.750,1251791.398,14.610,1566758.195,14.1315,1.489696e+06
3,000001.SZ,20200821,14.71,14.71,14.32,14.45,14.59,-0.14,-0.9596,1234517.33,1787278.581,14.896,1721507.186,14.559,1633584.344,14.0705,1.537756e+06
4,000001.SZ,20200820,15.01,15.14,14.53,14.59,15.10,-0.51,-3.3775,1333801.62,1962605.013,14.900,1695246.884,14.484,1609000.448,14.0230,1.567574e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,000001.SZ,20181114,10.53,10.55,10.42,10.44,10.54,-0.10,-0.9488,774239.85,810357.748,10.596,1038063.304,NaN,NaN,NaN,NaN
435,000001.SZ,20181113,10.41,10.65,10.38,10.54,10.56,-0.02,-0.1894,1072497.62,1126736.143,NaN,NaN,NaN,NaN,NaN,NaN
436,000001.SZ,20181112,10.46,10.58,10.38,10.56,10.55,0.01,0.0948,920333.88,965756.504,NaN,NaN,NaN,NaN,NaN,NaN
437,000001.SZ,20181109,10.71,10.73,10.40,10.55,10.89,-0.34,-3.1221,1629269.17,1716198.382,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
enddate = oneselect_stockdata[oneselect_stockdata['trade_date'] == analysis_data.iloc[0]['trade_date']]
for i in range(test_len):
    test_date = oneselect_stockdata.iloc[enddate.index[0]-1-i]
    if i == 2:
        break

In [11]:
one_average_line = average_line[average_line['trade_date'] == test_date['trade_date']].iloc[0]
# 在这套数据里,其实还可以比较成交量的关系.以后作为一个研究方向.
# 使用的均线也可以作为一个研究的目标. 目前只选用了 5, 10 ,20 三个均线.
# 这里的判定条件可以有三种: 目前先用第一个最宽的标准
criterion_para = 'criterion_1'
# 1, 只要最高价超过均线就算超过: criterion_1
if criterion_para == 'criterion_1':
    if one_average_line['ma5'] <= one_average_line['high'] and \
        one_average_line['ma10'] <= one_average_line['high'] and \
            one_average_line['ma20'] <= one_average_line['high'] :
        print('i get right one using criterion 1')
# 2, 必须开盘价超过均线才算超过: criterion_2
if criterion_para == 'criterion_2':
    if one_average_line['ma5'] <= one_average_line['open'] and \
        one_average_line['ma10'] <= one_average_line['open'] and \
            one_average_line['ma20'] <= one_average_line['open'] :
        print('i get right one using criterion 2')
# 3, 必须收盘价超过均线才算超过: criterion_3
if criterion_para == 'criterion_3':
    if one_average_line['ma5'] <= one_average_line['close'] and \
        one_average_line['ma10'] <= one_average_line['close'] and \
            one_average_line['ma20'] <= one_average_line['close'] :
        print('i get right one using criterion 3')

i get right one using criterion 1


### 好的, 以上做出了一个选择标准, 后面来设定上涨标准:

In [32]:
# test_date 是我目前获得, 并通过判断的数据. 这时候我们可以去判断后面是否产生了上涨.
# 一个简单粗暴的判定方法就是: 后几天的 pct_chg 的求和 是否大于某一个数字.
# 把观察天数的标准 设定为 upday_limit ,暂定为 5, 可以在后面的研究中更改.
upday_limit = 5
# 把 上涨 判定的标准设定为 uprate_limit = 4 , 寻找一周涨 4 % 已经不错了, 不要太贪心.
# 主要是测试的股票 5 天最后的收盘的上涨幅度也只有 4.6 所以先用 4 作为标准
uprate_limit = 4
######################################################################
day_Theselect = oneselect_stockdata[oneselect_stockdata['trade_date'] == test_date['trade_date']]
data_Theselect = oneselect_stockdata[day_Theselect.index[0] - upday_limit : day_Theselect.index[0]]
sum(data_Theselect['pct_chg']) > uprate_limit



True

## 现在对整个从 2019 年开始的数据进行测试:
1. 获得数据

In [38]:
# 研究从 20190101 开始
study_starttime = '20190101'
# 平均线选择的list,目前选择 5, 10 ,20
averagechose_list = [5, 10 , 20]

# 设定一个dict,用来存储所有数据.
# onedata_study 用来储存一组数据,其中包括
onedata_study = {}
# alldata_study 用来储存所有数据, keys 用 ts_code 做
alldata_study = {}

##########################################################
# 对所有 basic 中的股票进行研究:
for tscode_onestock in basic['ts_code']:
    print('i start to study :', tscode_onestock)
    onedata_study['ts_code'] = tscode_onestock
    ####################################
    # 这里一次性取得 平均线数据,比较方便.
    oneselect_stockdata = ts.pro_bar(api = pro, ts_code = tscode_onestock, start_date = study_starttime, adj = 'qfq', ma=averagechose_list)
    # oneselect_stockdata 记录着研究股票从 study_starttime : 20190101 开始的所有数据.
    onedata_study['oneselect_stockdata'] = oneselect_stockdata
    analysis_data = find_adjustperiod(oneselect_stockdata)
    # analysis_data 是记录符合要求的各个时间段数据
    onedata_study['analysis_data'] = analysis_data
    ##################################################
    # 接下来要对上面analysis_data 中的没一段数据进行分析


    ##################################################
    alldata_study[tscode_onestock] = onedata_study
    break
# 对数据逐笔进行检查,找到符合要求的时间段.


i start to study : 000001.SZ


In [39]:
oneselect_stockdata

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount,ma5,ma_v_5,ma10,ma_v_10,ma20,ma_v_20,trans_time
0,000001.SZ,20200826,14.6000,14.6100,14.2800,14.3700,14.6000,-0.2300,-1.5753,734117.72,1057274.169,14.4940,994041.150,14.656,1294990.030,14.2360,1.426989e+06,2020-08-26
1,000001.SZ,20200825,14.5600,14.6900,14.4600,14.6000,14.4600,0.1400,0.9682,748320.22,1090756.854,14.6400,1131403.228,14.657,1381259.428,14.1945,1.466262e+06,2020-08-25
2,000001.SZ,20200824,14.5000,14.7100,14.4100,14.4600,14.4500,0.0100,0.0692,919448.86,1338031.969,14.7500,1251791.398,14.610,1566758.195,14.1315,1.489696e+06,2020-08-24
3,000001.SZ,20200821,14.7100,14.7100,14.3200,14.4500,14.5900,-0.1400,-0.9596,1234517.33,1787278.581,14.8960,1721507.186,14.559,1633584.344,14.0705,1.537756e+06,2020-08-21
4,000001.SZ,20200820,15.0100,15.1400,14.5300,14.5900,15.1000,-0.5100,-3.3775,1333801.62,1962605.013,14.9000,1695246.884,14.484,1609000.448,14.0230,1.567574e+06,2020-08-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,000001.SZ,20190108,9.4657,9.4754,9.3586,9.3976,9.4754,-0.0778,-0.8211,402388.11,389247.795,9.2653,740831.820,NaN,NaN,NaN,NaN,2019-01-08
398,000001.SZ,20190107,9.5727,9.5824,9.3684,9.4754,9.4851,-0.0097,-0.1023,865687.66,841166.430,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-07
399,000001.SZ,20190104,8.9890,9.5532,8.9695,9.4851,9.0279,0.4572,5.0643,1481159.06,1422149.888,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-04
400,000001.SZ,20190103,8.9306,9.0765,8.9014,9.0279,8.9403,0.0876,0.9798,415537.95,384457.707,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-03


In [45]:
one_analysis_data = analysis_data[0]
one_analysis_data

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount,ma5,ma_v_5,ma10,ma_v_10,ma20,ma_v_20,trans_time
393,000001.SZ,20190114,9.9423,9.9715,9.7964,9.8353,9.9229,-0.0876,-0.8828,500443.59,507862.882,9.7303,780900.054,NaN,NaN,NaN,NaN,2019-01-14
394,000001.SZ,20190111,9.8353,9.9423,9.7770,9.9229,9.8256,0.0973,0.9903,696364.55,708001.802,9.6583,853948.868,NaN,NaN,NaN,NaN,2019-01-11
395,000001.SZ,20190110,9.6018,9.9229,9.5921,9.8256,9.6699,0.1557,1.6102,1071817.66,1079711.035,9.5707,1010907.770,NaN,NaN,NaN,NaN,2019-01-10
396,000001.SZ,20190109,9.4754,9.8061,9.4365,9.6699,9.3976,0.2723,2.8975,1233486.36,1229465.116,9.4112,879651.828,NaN,NaN,NaN,NaN,2019-01-09
397,000001.SZ,20190108,9.4657,9.4754,9.3586,9.3976,9.4754,-0.0778,-0.8211,402388.11,389247.795,9.2653,740831.820,NaN,NaN,NaN,NaN,2019-01-08
398,000001.SZ,20190107,9.5727,9.5824,9.3684,9.4754,9.4851,-0.0097,-0.1023,865687.66,841166.430,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-07


In [79]:
#逐一检验后续日期中是否有可以超过均线的天数.
# 检验的日期长度设定为: test_len
test_len = 10 # 检验后续三天的

##########################################################################
enddate = oneselect_stockdata[oneselect_stockdata['trade_date'] == one_analysis_data.iloc[0]['trade_date']]
for i in range(test_len):
    test_date = oneselect_stockdata.iloc[enddate.index[0]-1-i]
    one_judge_test = upstart_or_not(one_average_line = test_date)
    print(one_judge_test.keys())
    print(sum(one_judge_test.values()))
    #break


dict_keys(['20190115ma5judge', '20190115ma10judge', '20190115ma20judge'])
2
dict_keys(['20190116ma5judge', '20190116ma10judge', '20190116ma20judge'])
2
dict_keys(['20190117ma5judge', '20190117ma10judge', '20190117ma20judge'])
2
dict_keys(['20190118ma5judge', '20190118ma10judge', '20190118ma20judge'])
2
dict_keys(['20190121ma5judge', '20190121ma10judge', '20190121ma20judge'])
2
dict_keys(['20190122ma5judge', '20190122ma10judge', '20190122ma20judge'])
2
dict_keys(['20190123ma5judge', '20190123ma10judge', '20190123ma20judge'])
2
dict_keys(['20190124ma5judge', '20190124ma10judge', '20190124ma20judge'])
2
dict_keys(['20190125ma5judge', '20190125ma10judge', '20190125ma20judge'])
2
dict_keys(['20190128ma5judge', '20190128ma10judge', '20190128ma20judge'])
2


In [76]:
test_date

ts_code                 000001.SZ
trade_date               20190128
open                      10.7401
high                      10.8373
low                       10.5844
close                     10.6428
pre_close                 10.7011
change                    -0.0583
pct_chg                   -0.5448
vol                   1.03591e+06
amount                1.14185e+06
ma5                       10.3295
ma_v_5                     957161
ma10                      10.1982
ma_v_10                    858662
ma20                          NaN
ma_v_20                       NaN
trans_time    2019-01-28 00:00:00
Name: 383, dtype: object

In [50]:
oneselect_stockdata[oneselect_stockdata['trade_date'] == one_analysis_data.iloc[0]['trade_date']]

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount,ma5,ma_v_5,ma10,ma_v_10,ma20,ma_v_20,trans_time
393,000001.SZ,20190114,9.9423,9.9715,9.7964,9.8353,9.9229,-0.0876,-0.8828,500443.59,507862.882,9.7303,780900.054,NaN,NaN,NaN,NaN,2019-01-14


In [51]:
test_date

ts_code                 000001.SZ
trade_date               20190115
open                       9.8353
high                      10.0007
low                        9.8159
close                      9.9618
pre_close                  9.8353
change                     0.1265
pct_chg                    1.2862
vol                        542161
amount                     553027
ma5                        9.8431
ma_v_5                     808855
ma10                       9.5542
ma_v_10                    774843
ma20                          NaN
ma_v_20                       NaN
trans_time    2019-01-15 00:00:00
Name: 392, dtype: object

In [78]:
# 目前认为这个 函数是有用的 要载入
# 这里写成一普遍使用的函数.
# 这里的测试数据 
one_average_line = test_date

# 在这套数据里,其实还可以比较成交量的关系.以后作为一个研究方向.
# 使用的均线也可以作为一个研究的目标. 目前只选用了 5, 10 ,20 三个均线.
# 这里的判定条件可以有三种: 目前先用第一个最宽的标准
# criterion_para 是一个重要的参数,用于表示判定标准.可以选择 high, open, close 三总选择
# 分别表示用这个区和平均去做比较.
# 1, 只要最高价超过均线就算超过: high
# 2, 必须开盘价超过均线才算超过: open
# 3, 必须收盘价超过均线才算超过: close

criterion_para = 'high'

##############################################
def upstart_or_not(one_average_line , averagechose_list = [5, 10 , 20], criterion_para = 'high'):
    ##############################################
    judge_list = {}
    for average_one in averagechose_list:
        col_one = 'ma' + str(average_one)
        onejudge = one_average_line[criterion_para] >= one_average_line[col_one]
        judge_list[one_average_line['trade_date'] + col_one + 'judge'] = onejudge
    return judge_list

one_judge_test = upstart_or_not(one_average_line)
one_judge_test

{'20190128ma5judge': True,
 '20190128ma10judge': True,
 '20190128ma20judge': False}

In [58]:
one_average_line

ts_code                 000001.SZ
trade_date               20190115
open                       9.8353
high                      10.0007
low                        9.8159
close                      9.9618
pre_close                  9.8353
change                     0.1265
pct_chg                    1.2862
vol                        542161
amount                     553027
ma5                        9.8431
ma_v_5                     808855
ma10                       9.5542
ma_v_10                    774843
ma20                          NaN
ma_v_20                       NaN
trans_time    2019-01-15 00:00:00
Name: 392, dtype: object